In [ ]:
import pandas as pd
# i did the cassandra driver installed - conda install cassandra-driver
from cassandra.cluster import Cluster

In [ ]:
# creating the cassandra cluster
cluster = Cluster()
session = cluster.connect()

In [ ]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [ ]:
# key space is nothing but a database
session.execute("""CREATE KEYSPACE m14 WITH REPLICATION = {
                'class':'SimpleStrategy',
                'replication_factor': 2 
}""")

In [ ]:
# we can see what all keyspaces are there in the system using this command
rows = session.execute("desc keyspaces")
for i in rows:
    print(f"{i[0]}")

In [ ]:
# creating the table
session.execute("""
CREATE TABLE IF NOT EXISTS m14.warehouse_inventory(
                sku INT,
                name TEXT,
                description TEXT,
                warehouse_num INT,
                PRIMARY KEY(sku, warehouse_num)
);
""")

In [ ]:
# creating adummy data
data = [
    (1, 'chocklate', 'manufactured by melody', 1234),
    (2, 'biscuits', 'made by parle', 1235),
    (3, 'icecream', 'manufacturer: coldstone', 1236),
    (4, 'waffles', 'made by yummy inc', 1237),
    (5, 'beans', 'black beans', 1238),
    (6, 'spinach', 'fresh leafy vegetable', 1239),
    (7, 'chicken', 'chicken breasts', 1239),
    (8, 'fish', 'frozen salmon', 1239)
]

# i created a dataframe for the dummy data
df = pd.DataFrame(data, columns=['sku', 'name',
                  'description', 'warehouse_num'])

In [ ]:
df

In [ ]:
# Inserting the dummy data from the pandas data frame
for row in data:
    session.execute(
        """
        INSERT INTO m14.warehouse_inventory (sku, name, description, warehouse_num)
        VALUES (%s, %s, %s, %s)
        """,
        (row[0], row[1], row[2], row[3])
    )

In [ ]:
# i have created an index on the warehouse_num column for faster data retrival
session.execute("CREATE INDEX ON m14.warehouse_inventory(warehouse_num)")

In [ ]:
# i wrote a query to find out all the records that are in the warehouse 1239
rows = session.execute(
    "select (sku, name, description, warehouse_num) from m14.warehouse_inventory where warehouse_num = 1239;")

In [ ]:
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")